In [ ]:
!pip install transformers datasets evaluate py7zr rouge_score torch accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.3 M

**Huggingface**  **login**

**Step 2: Preprocess the Datase**t

Next, we preprocess the dataset by tokenizing the input articles and summaries.



In [ ]:
from transformers import T5Tokenizer
from datasets import load_dataset

# Load the SAMSum dataset
dataset = load_dataset('samsum')

# Load the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Define the preprocessing function
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Tokenize summaries
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=150, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

...

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

**Step 3: Load the Model**

Load the T5 model for conditional generation.



In [ ]:
from transformers import T5ForConditionalGeneration

# Load the model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

**Step 4: Define Summarization Function**

Create a function to summarize texts using the model.

In [ ]:
def summarize_text(text, model, tokenizer, max_length=150, min_length=40, num_beams=4):
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, length_penalty=2.0, num_beams=num_beams, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

**Step 6: Fine-Tuning the Model**

Set up fine-tuning using the Trainer class.



In [21]:
import torch
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
import evaluate

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    logging_dir='./logs',
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],  # Select the 'train' split
    eval_dataset=tokenized_datasets['validation'],  # Select the 'validation' split
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.380300,0.344125
2,0.371600,0.336938
3,0.363600,0.332871
4,0.359200,0.332321
5,0.360700,0.331042


TrainOutput(global_step=18415, training_loss=0.3686716693398618, metrics={'train_runtime': 4166.624, 'train_samples_per_second': 17.679, 'train_steps_per_second': 4.42, 'total_flos': 9969277096427520.0, 'train_loss': 0.3686716693398618, 'epoch': 5.0})

**Step-7 model saving**

In [22]:
model.save_pretrained('fine-tuned-abstractive/')
tokenizer.save_pretrained('fine-tuned-abstractive/')

('fine-tuned-abstractive/tokenizer_config.json',
 'fine-tuned-abstractive/special_tokens_map.json',
 'fine-tuned-abstractive/spiece.model',
 'fine-tuned-abstractive/added_tokens.json')

In [23]:
from datasets import load_metric

In [24]:
rouge=load_metric('rouge')

In [25]:
from transformers import T5ForConditionalGeneration

# Load the fine-tuned model
local_model = T5ForConditionalGeneration.from_pretrained('fine-tuned-abstractive/')

def evaluate(data, idx, local_tokenizer): # Pass the tokenizer as an argument
    # Use 'dialogue' instead of 'document'
    inputs=local_tokenizer('summarization: '+data['dialogue'][idx],return_tensors='pt')
    summary=local_model.generate(inputs['input_ids'],max_length=64) # Use the loaded model
    summary=local_tokenizer.decode(summary[0],skip_special_tokens=True)
    return summary

In [30]:
# Assuming 'samsum_dataset' is your dataset object
predicted_summary = [evaluate(dataset['validation'], i, tokenizer) for i in range(10)]

In [31]:

# Change 'summary' to the correct key for the summaries in your dataset
scores=rouge.compute(references=dataset['validation']['dialogue'][:10],predictions=predicted_summary)

In [33]:
for key in scores:
    print(f"{key}: {scores[key].high}")

rouge1: Score(precision=0.8457111359643189, recall=0.3345209725873078, fmeasure=0.442208369480505)
rouge2: Score(precision=0.5562528510312466, recall=0.2222090926265939, fmeasure=0.28804964834435565)
rougeL: Score(precision=0.7565472428248429, recall=0.30642550565106536, fmeasure=0.39660653427677967)
rougeLsum: Score(precision=0.8323487168024253, recall=0.3467631676208023, fmeasure=0.4501571017787591)
